In [ ]:
import docx
import os
import sys
import mysql.connector
doc = docx.Document("basic.docx")
result = [p.text for p in doc.paragraphs]

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

trained_words = {}
def get_trained_source():
    global trained_words
    try:
        with open("trained_words.json") as json_file:
                trained_words = json.load(json_file)
        return True
    except:
        print("오류!", "소스파일이 존재하지 않습니다.")
        return False


def convertToBinaryData(filename):
    # Convert digital data to binary format
    with open(filename, 'rb') as file:
        binaryData = file.read()
    return binaryData


def insertBLOB(Type, biodataFile):
    print("Inserting BLOB into python_employee table")
    try:
        connection = mysql.connector.connect(host='localhost',
                                             database='pvteam',
                                             user='root',
                                             password='pv3306')

        cursor = connection.cursor()
        sql_insert_blob_query = """ INSERT INTO literature_search
                          (Type, Contents) VALUES (%s,%s)"""

        #empPicture = convertToBinaryData('basic.docx')
        file = convertToBinaryData(biodataFile)

        # Convert data into tuple format
        insert_blob_tuple = (Type, file)
        result = cursor.execute(sql_insert_blob_query, insert_blob_tuple)
        connection.commit()
        print("Image and file inserted successfully as a BLOB into python_employee table", result)

    except mysql.connector.Error as error:
        print("Failed inserting BLOB data into MySQL table {}".format(error))

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")
            
def write_file(data, filename):
    # Convert binary data to proper format and write it on Hard Disk
    with open(filename, 'wb') as file:
        file.write(data)


def readBLOB(Type, bioData):
    print("Reading BLOB data from python_employee table")

    try:
        connection = mysql.connector.connect(host='localhost',
                                             database='pvteam',
                                             user='root',
                                             password='pv3306')

        cursor = connection.cursor()
        sql_fetch_blob_query = """SELECT * from literature_search"""
        cursor.execute(sql_fetch_blob_query)
        record = cursor.fetchall()
        for row in record:
            print("Type =", row[0], )
            file = row[1]
            print("Storing biodata and bio-data on disk \n")
            write_file(file, bioData)

    except mysql.connector.Error as error:
        print("Failed to read BLOB data from MySQL table {}".format(error))

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

#insertBLOB('pro', 'pro.docx')
readBLOB('pro','test_pro.docx')
